# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `15* - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [2]:

import json

def range_score(val, config_file, range_name):
    """ int or float, str, str -> int
    val: The measurement as an integer value
    config_file: The name of a configuration file to use for scoring 
    range_name: the name of the variable we want to examine
    
    The function should return a score for this subject based on where their variable
    falls in the scoring rules.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    rules = config.get(range_name)
    
    for rule in rules:
        if val >= rule.get('min') and val < rule.get('max'):
            score = rule.get('points')
    
    ### END SOLUTION

    return score

def Creatinine_score(val, Renal_Failure, config_file):
    """ float, str, str -> int
    val: The measurement as an integer value
    Renal_Failure: if the subject has renal failure, acute or chronic
    config_file: The name of a configuration file to use for scoring 
    
    The function should return a creatinine score for this subject based on where their variable
    falls in the scoring rules.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    rules = config.get('Creatinine')
    rules = rules.get(Renal_Failure)
    
    
    for rule in rules:
        if val >= rule.get('min') and val < rule.get('max'):
            score = rule.get('points')
    
    ### END SOLUTION

    return score

def O2_score(val, PaO2, Aa_gradient, config_file):
    """ int, int, int, str -> int
    val: The FiO2 measurement as an integer value
    Aa_gradient: A-a gradient score of the subject
    PaO2: the PaO2 score of the subject
    config_file: The name of a configuration file to use for scoring 
    
    The function should return an Oxygenation score for this subject based on where their variable
    falls in the scoring rules.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    
    if val <50:
        rules = config.get('FiO2')
        rules = rules.get('Pa02')
        for rule in rules:
            if PaO2 >= rule.get('min') and PaO2 <= rule.get('max'):
                score = rule.get('points')
    else:
        rules = config.get('FiO2')
        rules = rules.get('A-a Gradient')
        for rule in rules:
            if Aa_gradient >= rule.get('min') and Aa_gradient <= rule.get('max'):
                score = rule.get('points')
    
    
    ### END SOLUTION

    return score

def Glasgow_Coma(val):
    """ int -> int
    val: The Glasgow Coma Scale measurement as an integer value

    
    The function should return a Glasgow Coma Scale score for this subject based on 15- Glasgow Coma Scale.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    score = 15-val
    
    ### END SOLUTION

    return score

def Organ_Failure_History_score(val, config_file):
    """ int, str -> int
    val: The Organ Failure History measurement as a string value
    config_file: The name of a configuration file to use for scoring 
    
    The function should return a score for this subject based on where their variable
    falls in the scoring rules.
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    
    rules = config.get('Organ Failure History')
    score = rules[val]
    
    
    ### END SOLUTION

    return score

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [3]:
config_file='apache.json'
#Organ Failure History
assert( Organ_Failure_History_score("Nonoperative", config_file)== 5 )
assert( Organ_Failure_History_score("Emergency", config_file)== 5 )
assert( Organ_Failure_History_score("Elective", config_file)== 2 )
#Age
assert( range_score(46, config_file, 'Age')== 2 )
assert( range_score(73, config_file, 'Age')== 5 )
assert( range_score(54, config_file, 'Age')== 2 )
#Temperature
assert( range_score(32, config_file, 'Temperature')== 2 )
assert( range_score(37, config_file, 'Temperature')== 0 )
assert( range_score(40, config_file, 'Temperature')== 3 )
#pH
assert( range_score(7.29, config_file, 'pH')== 2 )
assert( range_score(7.46, config_file, 'pH')== 0 )
assert( range_score(7.18, config_file, 'pH')== 3 )
#Heart rate
assert( range_score(74, config_file, 'Heart Rate')== 0 )
assert( range_score(16, config_file, 'Heart Rate')== 4 )
assert( range_score(182, config_file, 'Heart Rate')== 4 )
#Respiratory rate
assert( range_score(47, config_file, 'Respiratory Rate')== 3 )
assert( range_score(12, config_file, 'Respiratory Rate')== 0 )
assert( range_score(27, config_file, 'Respiratory Rate')== 1 )
#Sodium
assert( range_score(158, config_file, 'Sodium')== 2 )
assert( range_score(132, config_file, 'Sodium')== 0 )
assert( range_score(113, config_file, 'Sodium')== 3 )
#Potassium
assert( range_score(0.6, config_file, 'Potassium')== 4 )
assert( range_score(7.6, config_file, 'Potassium')== 4 )
assert( range_score(8, config_file, 'Potassium')== 4 )
#Creatinine
assert( Creatinine_score(3.6, "Chronic Renal Failure", config_file)== 4 )
assert( Creatinine_score(1.8, "Chronic Renal Failure", config_file)== 2 )
assert( Creatinine_score(3.2, "Chronic Renal Failure", config_file)== 3 )
#Hematocrit
assert( range_score(39, config_file, 'Hematocrit')== 0 )
assert( range_score(50, config_file, 'Hematocrit')== 2 )
assert( range_score(9, config_file, 'Hematocrit')== 4 )
#White Blood Count
assert( range_score(22, config_file, 'White Blood Count')== 2 )
assert( range_score(32, config_file, 'White Blood Count')== 2 )
assert( range_score(44, config_file, 'White Blood Count')== 4 )
#FiO2
assert( O2_score(61, 458, 59, config_file)== 0 )
assert( O2_score(66, 59, 482, config_file)== 3 )
assert( O2_score(29, 69, 441, config_file)== 1 )
#Glasgow_Coma
assert(Glasgow_Coma(12)== 3)
assert(Glasgow_Coma(3)== 12)
assert(Glasgow_Coma(7)== 8)

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient




In [4]:
def apache_score(Organ_Failure_History, Age, Temperature, pH, Heart_rate, Respiratory_rate, Sodium,
                 Potassium, Creatinine, Acute_renal_failure, Hematocrit, White_Blood_Count, Glasgow_Coma_Scale,
                 FiO2, PaO2, Aa_gradient, config_file):
    """ see below -> int
    Organ_Failure_History:str: Surgical intervention for organ failure
    Age:int: age of individual
    Temperature:int: Rectal temperature, °C
    pH:float: Arterial pH
    Heart_rate:int: Heart rate, beats per minute
    Respiratory_rate:int: Respiratory rate, breaths per minute
    Sodium:int: Serum sodium, mmol/L
    Potassium:float: Serum potassium, mmol/L
    Creatinine:float: Serum creatinine, mg/100 mL
    Acute_renal_failure:str: the stage of renal failure
    Hematocrit:int: Hematocrit, %
    White_Blood_Count:int: White blood count, total/cubic mm in 1000's
    Glasgow_Coma_Scale:int: Coma severity based on Eye (4), Verbal (5), and Motor (6) criteria 
    FiO2:int: fraction of inspired oxygen
    PaO2:int:The partial pressure of oxygen
    Aa_gradient:int:Assesses for degree of shunting and V/Q mismatch.
    config_file:str: the file that contains the scoring rules
    
    The function utilizes the previously created functions in this file to produce the final score of the subject.
    """
    score= Organ_Failure_History_score(Organ_Failure_History, config_file)+\
        range_score(Age, config_file, 'Age')+\
        range_score(Temperature, config_file, 'Temperature')+\
        range_score(pH, config_file, 'pH')+\
        range_score(Heart_rate, config_file, 'Heart Rate')+\
        range_score(Respiratory_rate, config_file, 'Respiratory Rate')+\
        range_score(Sodium, config_file, 'Sodium')+\
        range_score(Potassium, config_file, 'Potassium')+\
        Creatinine_score(Creatinine, Acute_renal_failure, config_file)+\
        range_score(Hematocrit, config_file, 'Hematocrit')+\
        range_score(White_Blood_Count, config_file, 'White Blood Count')+\
        Glasgow_Coma(Glasgow_Coma_Scale)+\
        O2_score(FiO2, PaO2, Aa_gradient, config_file)
    
    return score


### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [5]:
config_file = "apache.json"
assert(apache_score("Emergency",6,27,7.3,174,10,170,7.8,1.6,"Chronic Renal Failure",
                    3,28,13,61,59,458,config_file)==35)
assert(apache_score("None",	66,	28,	7.9, 65, 3,	41,	1.4, 1.8, "Chronic Renal Failure",	
                    69,	21,	5,	35,	52,	426,config_file)==49)
assert(apache_score("Nonoperative",	58,	1,	7.2, 70, 35, 9,	8,	3.2, "Acute Renal Failure",	
                    48,	3,	5,	27,	65,	199,config_file)==44)


---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [6]:
import pandas as pd

data_file = pd.read_csv('https://hds5210-2020.s3.amazonaws.com/TestPatients.csv')

def test_data(data_file, config_file):
    """ str, str ->[int,int]
    data_file: the test data to run through the scoring function
    config_file: the scoring rules to be passed to subsquent functions
    
    This function is designed to produce a list of Apache II scores in the same order 
    as the inputs passed to it
    """ 
    data = pd.read_csv(data_file)
    scores=[]
    for ind in data.index: 
        score=(apache_score(data['Organ Failure History'][ind], 
              data['Age'][ind],
              data['Temperature'][ind],
              data['pH'][ind],
              data['Heart Rate'][ind],
              data['Respiratory Rate'][ind],
              data['Sodium'][ind],
              data['Potassium'][ind],
              data['Creatinine'][ind],
              data['Acute Renal Failure'][ind],
              data['Hematocrit'][ind],
              data['White Blood Count'][ind],
              data['Glasgow Coma Scale'][ind],
              data['FiO2'][ind],
              data['PaO2'][ind],
              data['A-a Gradient'][ind],
              config_file
                    ))
        scores.append(score)
    return scores
        



In [7]:
test_data('https://hds5210-2020.s3.amazonaws.com/TestPatients.csv', 'apache.json')

[35,
 31,
 47,
 34,
 44,
 35,
 31,
 49,
 40,
 48,
 42,
 43,
 32,
 41,
 42,
 49,
 37,
 37,
 38,
 43,
 41,
 31,
 38,
 30,
 41,
 41,
 34,
 46,
 40,
 47,
 36,
 43,
 41,
 46,
 44,
 40,
 39,
 37,
 41,
 30,
 46,
 30,
 41,
 44,
 35,
 36,
 40,
 40,
 30,
 50,
 52,
 43,
 46,
 34,
 33,
 42,
 41,
 31,
 46,
 46,
 34,
 36,
 33,
 38,
 26,
 29,
 46,
 25,
 40,
 38,
 39,
 34,
 39,
 53,
 44,
 49,
 37,
 36,
 36,
 51,
 33,
 36,
 43,
 41,
 24,
 50,
 29,
 40,
 36,
 50,
 29,
 37,
 34,
 45,
 34,
 40,
 37,
 37,
 47,
 31,
 33,
 50,
 28,
 37,
 33,
 44,
 40,
 38,
 40,
 31,
 44,
 37,
 44,
 47,
 25,
 34,
 32,
 41,
 38,
 34,
 38,
 39,
 41,
 34,
 29,
 44,
 22,
 40,
 34,
 43,
 40,
 31,
 35,
 33,
 27,
 44,
 52,
 40,
 53,
 40,
 40,
 48,
 31,
 42,
 49,
 43,
 35,
 31,
 36,
 42,
 34,
 31,
 30,
 28,
 38,
 47,
 32,
 47,
 31,
 41,
 40,
 33,
 35,
 42,
 49,
 55,
 51,
 37,
 53,
 29,
 33,
 46,
 36,
 32,
 33,
 33,
 42,
 32,
 41,
 35,
 39,
 40,
 40,
 35,
 28,
 33,
 42,
 29,
 30,
 39,
 34,
 46,
 30,
 37,
 31,
 37,
 41,
 47,
 45,
 29,


### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


## Verification

In [20]:
answers = pd.read_csv('https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv')
answer =answers.TOTAL.tolist()
answer =[int(i) for i in answer]

df = pd.DataFrame({'My Answers': test_data('https://hds5210-2020.s3.amazonaws.com/TestPatients.csv', 'apache.json'),
                   'Answer Key': answer})
good_job=[]
for ind in df.index:
    if df['My Answers'][ind]!=df['Answer Key'][ind]:
        print([ind, df['My Answers'][ind] , df['Answer Key'][ind], (df['My Answers'][ind] - df['Answer Key'][ind])])
    if df['My Answers'][ind]==df['Answer Key'][ind]:
        good_job.append("good job")
        
if len(good_job)==len(df.index):
    print("All is correct")
    print("YOU DID IT!! GREAT JOB!!!")

All is correct
YOU DID IT!! GREAT JOB!!!
